# ⚔️ t-test 삼총사: 상황별 올바른 선택

## 📖 세 명의 영웅

Student's t-분포를 발견한 후, 통계학자들은 **서로 다른 상황**에 맞는 세 가지 t-검정을 개발했습니다. 마치 **삼총사**처럼, 각각은 고유한 임무와 특성을 가지고 있습니다! ⚔️✨

### 🎭 삼총사를 소개합니다

1. **아토스 (One-sample t-test)** 🎯
   - **임무**: "우리 그룹이 특정 기준을 만족하는가?"
   - **특기**: 단일 그룹을 알려진 값과 비교
   - **대표 질문**: "우리 제품의 평균 무게가 500g인가?"

2. **포르토스 (Independent t-test)** ⚖️
   - **임무**: "두 독립적인 그룹이 정말 다른가?"
   - **특기**: 서로 다른 두 그룹 비교
   - **대표 질문**: "남성과 여성의 평균 키가 다른가?"

3. **아라미스 (Paired t-test)** 🔄
   - **임무**: "동일한 대상에서 변화가 있었는가?"
   - **특기**: 전후 비교, 짝지어진 데이터 분석
   - **대표 질문**: "다이어트 전후 체중 차이가 있는가?"

---

## 🎯 학습 목표

이 노트북에서 우리는:
1. **세 가지 t-검정의 차이점**을 명확히 이해합니다
2. **언제 어떤 검정을 사용할지** 판단하는 기준을 배웁니다
3. **실제 데이터**로 각 검정을 직접 수행해봅니다
4. **효과크기(Cohen's d)**와 **검정력** 개념을 학습합니다
5. **올바른 해석 방법**을 익힙니다

In [ ]:
print("⚔️ t-test 삼총사와의 만남을 시작합니다!")
print("🎭 각 영웅의 특별한 능력을 확인해보세요!")

## 🎯 1. 아토스 (One-sample t-test): 기준과의 대결

### 📋 아토스의 프로필
- **정체성**: 단일 표본 t-검정
- **사명**: 하나의 그룹이 알려진 기준값과 다른지 확인
- **무기**: 표본평균 vs 모집단 평균

### 🎪 언제 아토스를 부를까?

- 제품 품질 관리: "평균 무게가 규격에 맞나?"
- 학습 효과 측정: "우리 반 평균이 전국 평균과 다른가?"
- 의료 기준 검사: "환자의 수치가 정상 범위인가?"
- 성능 벤치마크: "우리 알고리즘이 기준 성능에 도달했나?"

### 🔢 아토스의 무술 (수학적 공식)

**가설 설정**:
- H₀: μ = μ₀ (표본 평균이 기준값과 같다)
- H₁: μ ≠ μ₀ (표본 평균이 기준값과 다르다)

**검정통계량**:
$$t = \\frac{\\bar{X} - \\mu_0}{\\frac{s}{\\sqrt{n}}} \\sim t_{n-1}$$

여기서:
- $\\bar{X}$: 표본평균
- $\\mu_0$: 비교하고자 하는 기준값
- $s$: 표본표준편차
- $n$: 표본크기

In [ ]:
# 🎯 아토스의 모험: One-sample t-test 시뮬레이션
class OnesampleTTestHero:
    def __init__(self, name=\"아토스\"):
        self.name = name
        self.mission = \"단일 그룹을 기준값과 비교하기\"
    
    def perform_test(self, data, mu0, alpha=0.05, alternative='two-sided'):
        \"\"\"One-sample t-test 수행\"\"\"
        
        n = len(data)
        sample_mean = np.mean(data)
        sample_std = np.std(data, ddof=1)
        
        # t-통계량 계산
        t_statistic = (sample_mean - mu0) / (sample_std / np.sqrt(n))
        
        # 자유도
        df = n - 1
        
        # p-값 계산
        if alternative == 'two-sided':
            p_value = 2 * (1 - stats.t.cdf(abs(t_statistic), df))
            critical_t = stats.t.ppf(1 - alpha/2, df)
            reject_condition = abs(t_statistic) > critical_t
        elif alternative == 'greater':
            p_value = 1 - stats.t.cdf(t_statistic, df)
            critical_t = stats.t.ppf(1 - alpha, df)
            reject_condition = t_statistic > critical_t
        else:  # 'less'
            p_value = stats.t.cdf(t_statistic, df)
            critical_t = stats.t.ppf(alpha, df)
            reject_condition = t_statistic < critical_t
        
        # 신뢰구간 (양측)
        margin_error = stats.t.ppf(1 - alpha/2, df) * (sample_std / np.sqrt(n))
        ci_lower = sample_mean - margin_error
        ci_upper = sample_mean + margin_error
        
        # 효과크기 (Cohen's d)
        cohens_d = (sample_mean - mu0) / sample_std
        
        return {
            'sample_size': n,
            'sample_mean': sample_mean,
            'sample_std': sample_std,
            'mu0': mu0,
            't_statistic': t_statistic,
            'df': df,
            'p_value': p_value,
            'critical_t': critical_t,
            'reject_null': reject_condition,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'cohens_d': cohens_d,
            'alpha': alpha,
            'alternative': alternative
        }
    
    def visualize_test(self, result, data):
        \"\"\"One-sample t-test 결과 시각화\"\"\"
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                '1. 데이터 분포와 기준값',
                '2. t-분포와 검정통계량',
                '3. 신뢰구간',
                '4. 효과크기 해석'
            ],
            specs=[
                [{'type': 'histogram'}, {'type': 'scatter'}],
                [{'type': 'scatter'}, {'type': 'bar'}]
            ]
        )
        
        # 1. 데이터 분포
        fig.add_trace(
            go.Histogram(
                x=data,
                nbinsx=20,
                name='데이터',
                marker_color='lightblue',
                opacity=0.7
            ),
            row=1, col=1
        )
        
        # 표본평균 선
        fig.add_vline(
            x=result['sample_mean'],
            line_dash=\"solid\",
            line_color=\"blue\",
            annotation_text=f\"표본평균: {result['sample_mean']:.2f}\",
            row=1, col=1
        )
        
        # 기준값 선
        fig.add_vline(
            x=result['mu0'],
            line_dash=\"dash\",
            line_color=\"red\",
            annotation_text=f\"기준값: {result['mu0']:.2f}\",
            row=1, col=1
        )
        
        # 2. t-분포와 검정통계량
        t_range = np.linspace(-4, 4, 200)
        t_pdf = stats.t.pdf(t_range, result['df'])
        
        fig.add_trace(
            go.Scatter(
                x=t_range,
                y=t_pdf,
                mode='lines',
                name=f't-분포 (df={result[\"df\"]})',
                line=dict(color='blue', width=2)
            ),
            row=1, col=2
        )
        
        # 검정통계량 위치
        fig.add_vline(
            x=result['t_statistic'],
            line_dash=\"solid\",
            line_color=\"red\" if result['reject_null'] else \"green\",
            annotation_text=f\"t = {result['t_statistic']:.3f}\",
            row=1, col=2
        )
        
        # 임계값 표시
        if result['alternative'] == 'two-sided':
            for crit in [-result['critical_t'], result['critical_t']]:
                fig.add_vline(
                    x=crit,
                    line_dash=\"dash\",
                    line_color=\"orange\",
                    row=1, col=2
                )
        
        # 3. 신뢰구간
        fig.add_trace(
            go.Scatter(
                x=[result['ci_lower'], result['ci_upper']],
                y=[1, 1],
                mode='lines+markers',
                name='95% 신뢰구간',
                line=dict(color='green', width=5),
                marker=dict(size=10)
            ),
            row=2, col=1
        )
        
        # 표본평균과 기준값 표시
        fig.add_trace(
            go.Scatter(
                x=[result['sample_mean']],
                y=[1.1],
                mode='markers',
                name='표본평균',
                marker=dict(color='blue', size=12, symbol='diamond')
            ),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Scatter(
                x=[result['mu0']],
                y=[0.9],
                mode='markers',
                name='기준값',
                marker=dict(color='red', size=12, symbol='x')
            ),
            row=2, col=1
        )
        
        # 4. 효과크기 해석
        cohens_d = abs(result['cohens_d'])
        if cohens_d < 0.2:
            effect_size = '매우 작음'
            color = 'lightgray'
        elif cohens_d < 0.5:
            effect_size = '작음'
            color = 'yellow'
        elif cohens_d < 0.8:
            effect_size = '중간'
            color = 'orange'
        else:
            effect_size = '큼'
            color = 'red'
        
        fig.add_trace(
            go.Bar(
                x=['Cohen\'s d'],
                y=[cohens_d],
                name='효과크기',
                marker_color=color,
                text=[f'{cohens_d:.3f}<br>({effect_size})'],
                textposition='auto'
            ),
            row=2, col=2
        )
        
        # 레이아웃 업데이트
        decision = \"귀무가설 기각\" if result['reject_null'] else \"귀무가설 채택\"
        fig.update_layout(
            title=f'🎯 {self.name}의 결과: {decision} (p = {result[\"p_value\"]:.4f})',
            height=800
        )
        
        return fig

# 아토스 인스턴스 생성
athos = OnesampleTTestHero(\"아토스\")

# 예제 데이터: 스마트폰 배터리 수명 (시간)
# 제조사 공칭 수명: 24시간
np.random.seed(42)
battery_life = np.random.normal(23.2, 2.1, 25)  # 실제로는 조금 짧음

print(\"🔋 사례: 스마트폰 배터리 수명 검사\")
print(f\"📊 공칭 배터리 수명: 24시간\")
print(f\"🔬 검사한 배터리 수: {len(battery_life)}개\")
print(f\"📈 측정된 평균 수명: {np.mean(battery_life):.2f}시간\")
print(\"\")

# 아토스의 검정 수행
athos_result = athos.perform_test(battery_life, mu0=24, alpha=0.05)

# 결과 시각화
fig_athos = athos.visualize_test(athos_result, battery_life)
fig_athos.show()

# 결과 해석
print(f\"\"\"\n🎯 {athos.name}의 보고서:\n
📊 검정 결과:\n
   • 표본 크기: {athos_result['sample_size']}\n
   • 표본 평균: {athos_result['sample_mean']:.3f}시간\n
   • 표본 표준편차: {athos_result['sample_std']:.3f}시간\n
   • t-통계량: {athos_result['t_statistic']:.3f}\n
   • p-값: {athos_result['p_value']:.4f}\n
\n
🎯 95% 신뢰구간: [{athos_result['ci_lower']:.2f}, {athos_result['ci_upper']:.2f}]시간\n
📏 효과크기 (Cohen's d): {athos_result['cohens_d']:.3f}\n
\n
{'🚨 결론: 배터리 수명이 공칭값보다 유의하게 짧습니다!' if athos_result['reject_null'] else '✅ 결론: 배터리 수명이 공칭값과 차이가 없습니다.'}\n
💡 해석: {'실제 배터리 수명이 24시간보다 짧을 가능성이 높습니다.' if athos_result['reject_null'] else '실제 배터리 수명이 24시간과 크게 다르지 않습니다.'}\n
\"\"\")

## ⚖️ 2. 포르토스 (Independent t-test): 두 군단의 대결

### 📋 포르토스의 프로필
- **정체성**: 독립표본 t-검정
- **사명**: 서로 독립적인 두 그룹의 평균이 다른지 확인
- **무기**: 그룹 A 평균 vs 그룹 B 평균

### 🎪 언제 포르토스를 부를까?

- A/B 테스트: "신버전이 구버전보다 나은가?"
- 성별 차이 연구: "남성과 여성의 평균 임금이 다른가?"
- 치료법 비교: "약물 A가 약물 B보다 효과적인가?"
- 지역별 비교: "서울과 부산의 집값이 다른가?"

### 🔢 포르토스의 무술 (수학적 공식)

**가설 설정**:
- H₀: μ₁ = μ₂ (두 그룹의 평균이 같다)
- H₁: μ₁ ≠ μ₂ (두 그룹의 평균이 다르다)

**검정통계량 (등분산 가정)**:
$$t = \\frac{\\bar{X_1} - \\bar{X_2}}{s_p \\sqrt{\\frac{1}{n_1} + \\frac{1}{n_2}}} \\sim t_{n_1+n_2-2}$$

여기서 **합동분산(pooled variance)**:
$$s_p^2 = \\frac{(n_1-1)s_1^2 + (n_2-1)s_2^2}{n_1+n_2-2}$$

### ⚠️ 포르토스의 조건
1. **독립성**: 두 그룹이 서로 독립적이어야 함
2. **정규성**: 각 그룹이 (근사적으로) 정규분포를 따름
3. **등분산성**: 두 그룹의 분산이 같아야 함 (Student's t-test의 경우)

In [ ]:
# ⚖️ 포르토스의 모험: Independent t-test 시뮬레이션
class IndependentTTestHero:
    def __init__(self, name=\"포르토스\"):
        self.name = name
        self.mission = \"두 독립적인 그룹 비교하기\"
    
    def perform_test(self, group1, group2, alpha=0.05, equal_var=True, alternative='two-sided'):
        \"\"\"Independent t-test 수행\"\"\"
        
        n1, n2 = len(group1), len(group2)
        mean1, mean2 = np.mean(group1), np.mean(group2)
        std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
        
        if equal_var:
            # Student's t-test (등분산 가정)
            # 합동분산 계산
            pooled_var = ((n1-1)*std1**2 + (n2-1)*std2**2) / (n1+n2-2)
            pooled_std = np.sqrt(pooled_var)
            
            # 표준오차
            se = pooled_std * np.sqrt(1/n1 + 1/n2)
            
            # 자유도
            df = n1 + n2 - 2
            
        else:
            # Welch's t-test (등분산 가정 안함)
            se = np.sqrt(std1**2/n1 + std2**2/n2)
            
            # Welch-Satterthwaite 자유도
            df = (std1**2/n1 + std2**2/n2)**2 / ((std1**2/n1)**2/(n1-1) + (std2**2/n2)**2/(n2-1))
        
        # t-통계량 계산
        t_statistic = (mean1 - mean2) / se
        
        # p-값 계산
        if alternative == 'two-sided':
            p_value = 2 * (1 - stats.t.cdf(abs(t_statistic), df))
            critical_t = stats.t.ppf(1 - alpha/2, df)
            reject_condition = abs(t_statistic) > critical_t
        elif alternative == 'greater':
            p_value = 1 - stats.t.cdf(t_statistic, df)
            critical_t = stats.t.ppf(1 - alpha, df)
            reject_condition = t_statistic > critical_t
        else:  # 'less'
            p_value = stats.t.cdf(t_statistic, df)
            critical_t = stats.t.ppf(alpha, df)
            reject_condition = t_statistic < critical_t
        
        # 신뢰구간 (평균 차이에 대한)
        margin_error = stats.t.ppf(1 - alpha/2, df) * se
        mean_diff = mean1 - mean2
        ci_lower = mean_diff - margin_error
        ci_upper = mean_diff + margin_error
        
        # 효과크기 (Cohen's d)
        if equal_var:
            cohens_d = (mean1 - mean2) / pooled_std
        else:
            # 등분산이 아닐 때는 가중평균 사용
            pooled_std_approx = np.sqrt(((n1-1)*std1**2 + (n2-1)*std2**2) / (n1+n2-2))
            cohens_d = (mean1 - mean2) / pooled_std_approx
        
        # Levene's test for equal variances
        levene_stat, levene_p = stats.levene(group1, group2)
        
        return {
            'n1': n1, 'n2': n2,
            'mean1': mean1, 'mean2': mean2,
            'std1': std1, 'std2': std2,
            'mean_diff': mean_diff,
            't_statistic': t_statistic,
            'df': df,
            'p_value': p_value,
            'critical_t': critical_t,
            'reject_null': reject_condition,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'cohens_d': cohens_d,
            'equal_var': equal_var,
            'levene_stat': levene_stat,
            'levene_p': levene_p,
            'alpha': alpha,
            'alternative': alternative
        }
    
    def visualize_test(self, result, group1, group2, group1_name='그룹 1', group2_name='그룹 2'):
        \"\"\"Independent t-test 결과 시각화\"\"\"
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                '1. 두 그룹 분포 비교',
                '2. 박스 플롯 비교',
                '3. t-분포와 검정통계량',
                '4. 평균 차이 신뢰구간'
            ],
            specs=[
                [{'type': 'histogram'}, {'type': 'box'}],
                [{'type': 'scatter'}, {'type': 'scatter'}]
            ]
        )
        
        # 1. 히스토그램 비교
        fig.add_trace(
            go.Histogram(
                x=group1,
                nbinsx=20,
                name=group1_name,
                marker_color='lightblue',
                opacity=0.6
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Histogram(
                x=group2,
                nbinsx=20,
                name=group2_name,
                marker_color='lightcoral',
                opacity=0.6
            ),
            row=1, col=1
        )
        
        # 2. 박스 플롯
        fig.add_trace(
            go.Box(
                y=group1,
                name=group1_name,
                marker_color='lightblue'
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Box(
                y=group2,
                name=group2_name,
                marker_color='lightcoral'
            ),
            row=1, col=2
        )
        
        # 3. t-분포
        t_range = np.linspace(-4, 4, 200)
        t_pdf = stats.t.pdf(t_range, result['df'])
        
        fig.add_trace(
            go.Scatter(
                x=t_range,
                y=t_pdf,
                mode='lines',
                name=f't-분포 (df={result[\"df\"]:.1f})',
                line=dict(color='blue', width=2)
            ),
            row=2, col=1
        )
        
        # 검정통계량 표시
        fig.add_vline(
            x=result['t_statistic'],
            line_dash=\"solid\",
            line_color=\"red\" if result['reject_null'] else \"green\",
            annotation_text=f\"t = {result['t_statistic']:.3f}\",
            row=2, col=1
        )
        
        # 4. 평균 차이 신뢰구간
        fig.add_trace(
            go.Scatter(
                x=[result['ci_lower'], result['ci_upper']],
                y=[1, 1],
                mode='lines+markers',
                name='95% 신뢰구간',
                line=dict(color='green', width=5),
                marker=dict(size=10)
            ),
            row=2, col=2
        )
        
        # 평균 차이 점 추가
        fig.add_trace(
            go.Scatter(
                x=[result['mean_diff']],
                y=[1.1],
                mode='markers',
                name='평균 차이',
                marker=dict(color='red', size=12, symbol='diamond')
            ),
            row=2, col=2
        )
        
        # 차이 없음 선 (0)
        fig.add_vline(
            x=0,
            line_dash=\"dash\",
            line_color=\"black\",
            annotation_text=\"차이 없음\",
            row=2, col=2
        )
        
        # 레이아웃
        decision = \"유의한 차이 있음\" if result['reject_null'] else \"유의한 차이 없음\"
        test_type = \"Student's t-test\" if result['equal_var'] else \"Welch's t-test\"
        
        fig.update_layout(
            title=f'⚖️ {self.name}의 결과 ({test_type}): {decision}',
            height=800
        )
        
        return fig

# 포르토스 인스턴스 생성
porthos = IndependentTTestHero(\"포르토스\")

# 예제 데이터: 두 지역의 커피 가격 비교
np.random.seed(42)
seoul_coffee_price = np.random.normal(4200, 400, 30)  # 서울 커피 가격
busan_coffee_price = np.random.normal(3800, 350, 25)  # 부산 커피 가격

print(\"☕ 사례: 서울 vs 부산 커피 가격 비교\")
print(f\"📊 서울 커피숍 수: {len(seoul_coffee_price)}개\")
print(f\"📊 부산 커피숍 수: {len(busan_coffee_price)}개\")
print(f\"💰 서울 평균 가격: {np.mean(seoul_coffee_price):.0f}원\")
print(f\"💰 부산 평균 가격: {np.mean(busan_coffee_price):.0f}원\")
print(\"\")

# 등분산성 검정 먼저 수행
levene_stat, levene_p = stats.levene(seoul_coffee_price, busan_coffee_price)
equal_var = levene_p > 0.05  # p > 0.05이면 등분산 가정

print(f\"🔍 등분산성 검정 (Levene's test): p = {levene_p:.4f}\")
print(f\"📋 등분산 가정: {'성립' if equal_var else '위반'} → {'Student\'s' if equal_var else 'Welch\'s'} t-test 사용\")
print(\"\")

# 포르토스의 검정 수행
porthos_result = porthos.perform_test(
    seoul_coffee_price, busan_coffee_price, 
    alpha=0.05, equal_var=equal_var
)

# 결과 시각화
fig_porthos = porthos.visualize_test(
    porthos_result, seoul_coffee_price, busan_coffee_price,
    '서울', '부산'
)
fig_porthos.show()

# 결과 해석
print(f\"\"\"\n⚖️ {porthos.name}의 보고서:\n
📊 검정 결과:\n
   • 서울 표본: n={porthos_result['n1']}, 평균={porthos_result['mean1']:.0f}원, SD={porthos_result['std1']:.0f}원\n
   • 부산 표본: n={porthos_result['n2']}, 평균={porthos_result['mean2']:.0f}원, SD={porthos_result['std2']:.0f}원\n
   • 평균 차이: {porthos_result['mean_diff']:.0f}원\n
   • t-통계량: {porthos_result['t_statistic']:.3f}\n
   • 자유도: {porthos_result['df']:.1f}\n
   • p-값: {porthos_result['p_value']:.4f}\n
\n
🎯 평균 차이 95% 신뢰구간: [{porthos_result['ci_lower']:.0f}, {porthos_result['ci_upper']:.0f}]원\n
📏 효과크기 (Cohen's d): {porthos_result['cohens_d']:.3f}\n
\n
{'🚨 결론: 서울과 부산의 커피 가격에 유의한 차이가 있습니다!' if porthos_result['reject_null'] else '✅ 결론: 서울과 부산의 커피 가격에 유의한 차이가 없습니다.'}\n
💡 해석: {'서울이 부산보다 커피가 유의하게 비쌉니다.' if porthos_result['reject_null'] and porthos_result['mean_diff'] > 0 else '두 지역의 커피 가격이 비슷합니다.' if not porthos_result['reject_null'] else '부산이 서울보다 커피가 유의하게 비쌉니다.'}\n
\"\"\")

## 🔄 3. 아라미스 (Paired t-test): 변화의 탐지

### 📋 아라미스의 프로필
- **정체성**: 대응표본 t-검정
- **사명**: 동일한 대상에서 전후 변화가 있는지 확인
- **무기**: 차이값(difference)의 분석

### 🎪 언제 아라미스를 부를까?

- 치료 효과: "수술 전후 환자 상태가 개선되었나?"
- 학습 효과: "교육 전후 성적이 향상되었나?"
- 제품 개선: "업데이트 전후 성능이 달라졌나?"
- 시간 경과: "1년 전과 지금의 체중이 다른가?"

### 🔢 아라미스의 무술 (수학적 공식)

**핵심 아이디어**: 각 개체의 **차이값(difference)**을 계산하여 분석

$$D_i = X_{i,after} - X_{i,before}$$

**가설 설정**:
- H₀: μ_D = 0 (평균 차이가 0이다)
- H₁: μ_D ≠ 0 (평균 차이가 0이 아니다)

**검정통계량**:
$$t = \\frac{\\bar{D}}{\\frac{s_D}{\\sqrt{n}}} \\sim t_{n-1}$$

여기서:
- $\\bar{D}$: 차이값들의 평균
- $s_D$: 차이값들의 표준편차

### 🎯 아라미스 vs 포르토스

**언제 아라미스를 선택해야 할까?**

| 상황 | 아라미스 (Paired) | 포르토스 (Independent) |
|------|------------------|----------------------|
| **데이터 구조** | 같은 개체의 전후 측정 | 다른 개체들의 그룹 비교 |
| **예시** | 환자 10명의 수술 전후 | A그룹 vs B그룹 |
| **장점** | 개체간 변동 제거 | 설계가 간단 |
| **검정력** | 일반적으로 더 높음 | 상황에 따라 다름 |

In [ ]:
# 🔄 아라미스의 모험: Paired t-test 시뮬레이션
class PairedTTestHero:
    def __init__(self, name=\"아라미스\"):
        self.name = name
        self.mission = \"동일 개체의 전후 변화 탐지하기\"
    
    def perform_test(self, before, after, alpha=0.05, alternative='two-sided'):
        \"\"\"Paired t-test 수행\"\"\"
        
        # 차이값 계산
        differences = np.array(after) - np.array(before)
        
        n = len(differences)
        mean_diff = np.mean(differences)
        std_diff = np.std(differences, ddof=1)
        
        # t-통계량 계산
        t_statistic = mean_diff / (std_diff / np.sqrt(n))
        
        # 자유도
        df = n - 1
        
        # p-값 계산
        if alternative == 'two-sided':
            p_value = 2 * (1 - stats.t.cdf(abs(t_statistic), df))
            critical_t = stats.t.ppf(1 - alpha/2, df)
            reject_condition = abs(t_statistic) > critical_t
        elif alternative == 'greater':
            p_value = 1 - stats.t.cdf(t_statistic, df)
            critical_t = stats.t.ppf(1 - alpha, df)
            reject_condition = t_statistic > critical_t
        else:  # 'less'
            p_value = stats.t.cdf(t_statistic, df)
            critical_t = stats.t.ppf(alpha, df)
            reject_condition = t_statistic < critical_t
        
        # 신뢰구간 (차이의 평균에 대한)
        margin_error = stats.t.ppf(1 - alpha/2, df) * (std_diff / np.sqrt(n))
        ci_lower = mean_diff - margin_error
        ci_upper = mean_diff + margin_error
        
        # 효과크기 (Cohen's d for paired samples)
        cohens_d = mean_diff / std_diff
        
        # 상관계수 (before vs after)
        correlation = np.corrcoef(before, after)[0, 1]
        
        return {
            'n': n,
            'mean_before': np.mean(before),
            'mean_after': np.mean(after),
            'std_before': np.std(before, ddof=1),
            'std_after': np.std(after, ddof=1),
            'mean_diff': mean_diff,
            'std_diff': std_diff,
            'differences': differences,
            't_statistic': t_statistic,
            'df': df,
            'p_value': p_value,
            'critical_t': critical_t,
            'reject_null': reject_condition,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper,
            'cohens_d': cohens_d,
            'correlation': correlation,
            'alpha': alpha,
            'alternative': alternative
        }
    
    def visualize_test(self, result, before, after, before_name='Before', after_name='After'):
        \"\"\"Paired t-test 결과 시각화\"\"\"
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=[
                '1. 전후 비교 (개별 선)',
                '2. 차이값 분포',
                '3. 전후 산점도',
                '4. 차이값 신뢰구간'
            ],
            specs=[
                [{'type': 'scatter'}, {'type': 'histogram'}],
                [{'type': 'scatter'}, {'type': 'scatter'}]
            ]
        )
        
        # 1. 전후 비교 (각 개체별 선으로 연결)
        for i in range(len(before)):
            fig.add_trace(
                go.Scatter(
                    x=[before_name, after_name],
                    y=[before[i], after[i]],
                    mode='lines+markers',
                    line=dict(width=1, color='lightgray'),
                    marker=dict(size=4),
                    showlegend=False,
                    hovertemplate=f'개체 {i+1}: %{{y}}<extra></extra>'
                ),
                row=1, col=1
            )
        
        # 평균선 추가
        fig.add_trace(
            go.Scatter(
                x=[before_name, after_name],
                y=[result['mean_before'], result['mean_after']],
                mode='lines+markers',
                line=dict(width=4, color='red'),
                marker=dict(size=10, color='red'),
                name='평균'
            ),
            row=1, col=1
        )
        
        # 2. 차이값 히스토그램
        fig.add_trace(
            go.Histogram(
                x=result['differences'],
                nbinsx=15,
                name='차이값',
                marker_color='lightgreen',
                opacity=0.7
            ),
            row=1, col=2
        )
        
        # 차이값 평균 선
        fig.add_vline(
            x=result['mean_diff'],
            line_dash=\"solid\",
            line_color=\"red\",
            annotation_text=f\"평균 차이: {result['mean_diff']:.2f}\",
            row=1, col=2
        )
        
        # 차이 없음 선 (0)
        fig.add_vline(
            x=0,
            line_dash=\"dash\",
            line_color=\"black\",
            annotation_text=\"차이 없음\",
            row=1, col=2
        )
        
        # 3. 전후 산점도 (상관관계 확인)
        fig.add_trace(
            go.Scatter(
                x=before,
                y=after,
                mode='markers',
                name='개별 데이터',
                marker=dict(size=8, color='blue', opacity=0.6)
            ),
            row=2, col=1
        )
        
        # 대각선 (변화 없음)
        min_val = min(min(before), min(after))
        max_val = max(max(before), max(after))
        fig.add_trace(
            go.Scatter(
                x=[min_val, max_val],
                y=[min_val, max_val],
                mode='lines',
                line=dict(dash='dash', color='gray'),
                name='변화 없음'
            ),
            row=2, col=1
        )
        
        # 4. 차이값 신뢰구간
        fig.add_trace(
            go.Scatter(
                x=[result['ci_lower'], result['ci_upper']],
                y=[1, 1],
                mode='lines+markers',
                name='95% 신뢰구간',
                line=dict(color='green', width=5),
                marker=dict(size=10)
            ),
            row=2, col=2
        )
        
        # 평균 차이 점
        fig.add_trace(
            go.Scatter(
                x=[result['mean_diff']],
                y=[1.1],
                mode='markers',
                name='평균 차이',
                marker=dict(color='red', size=12, symbol='diamond')
            ),
            row=2, col=2
        )
        
        # 차이 없음 선
        fig.add_vline(
            x=0,
            line_dash=\"dash\",
            line_color=\"black\",
            annotation_text=\"변화 없음\",
            row=2, col=2
        )
        
        # 레이아웃
        decision = \"유의한 변화 있음\" if result['reject_null'] else \"유의한 변화 없음\"
        
        fig.update_layout(
            title=f'🔄 {self.name}의 결과: {decision} (r = {result[\"correlation\"]:.3f})',
            height=800
        )
        
        # 축 레이블 업데이트
        fig.update_xaxes(title_text=before_name, row=2, col=1)
        fig.update_yaxes(title_text=after_name, row=2, col=1)
        
        return fig

# 아라미스 인스턴스 생성
aramis = PairedTTestHero(\"아라미스\")

# 예제 데이터: 다이어트 프로그램 전후 체중 변화
np.random.seed(42)
n_participants = 20

# 참가자들의 초기 체중 (개인차 반영)
initial_weights = np.random.normal(70, 10, n_participants)

# 다이어트 효과 (평균 3kg 감량, 개인차 있음)
weight_loss = np.random.normal(3, 1.5, n_participants)

# 다이어트 전후 체중
before_weight = initial_weights
after_weight = initial_weights - weight_loss  # 체중 감소

print(\"🏃‍♀️ 사례: 다이어트 프로그램 효과 분석\")
print(f\"👥 참가자 수: {n_participants}명\")
print(f\"⚖️ 다이어트 전 평균 체중: {np.mean(before_weight):.1f}kg\")
print(f\"⚖️ 다이어트 후 평균 체중: {np.mean(after_weight):.1f}kg\")
print(f\"📉 평균 체중 변화: {np.mean(after_weight - before_weight):.1f}kg\")
print(\"\")

# 아라미스의 검정 수행
aramis_result = aramis.perform_test(
    before_weight, after_weight, 
    alpha=0.05, alternative='less'  # 체중이 감소했는지 확인 (단측검정)
)

# 결과 시각화
fig_aramis = aramis.visualize_test(
    aramis_result, before_weight, after_weight,
    '다이어트 전', '다이어트 후'
)
fig_aramis.show()

# 결과 해석
print(f\"\"\"\n🔄 {aramis.name}의 보고서:\n
📊 검정 결과:\n
   • 참가자 수: {aramis_result['n']}명\n
   • 다이어트 전 평균: {aramis_result['mean_before']:.1f}kg (SD: {aramis_result['std_before']:.1f}kg)\n
   • 다이어트 후 평균: {aramis_result['mean_after']:.1f}kg (SD: {aramis_result['std_after']:.1f}kg)\n
   • 평균 체중 변화: {aramis_result['mean_diff']:.1f}kg\n
   • t-통계량: {aramis_result['t_statistic']:.3f}\n
   • p-값: {aramis_result['p_value']:.4f}\n
\n
🎯 체중 변화 95% 신뢰구간: [{aramis_result['ci_lower']:.1f}, {aramis_result['ci_upper']:.1f}]kg\n
📏 효과크기 (Cohen's d): {aramis_result['cohens_d']:.3f}\n
🔗 전후 상관계수: {aramis_result['correlation']:.3f}\n
\n
{'🎉 결론: 다이어트 프로그램이 유의한 체중 감소 효과가 있습니다!' if aramis_result['reject_null'] else '😔 결론: 다이어트 프로그램의 효과가 통계적으로 유의하지 않습니다.'}\n
💡 해석: {'참가자들이 유의하게 체중을 감량했습니다.' if aramis_result['reject_null'] else '체중 변화가 우연에 의한 것일 수 있습니다.'}\n
\"\"\")

## 📊 4. 효과크기와 검정력: 통계적 유의성을 넘어서

### 🤔 p-값만으로 충분할까?

통계적 유의성(p < 0.05)은 **차이가 존재한다**는 것을 알려주지만, **얼마나 중요한 차이인지**는 말해주지 않습니다.

### 📏 효과크기 (Effect Size): Cohen's d

**Cohen's d**는 두 평균 간의 차이를 **표준편차 단위**로 표현한 것입니다.

#### One-sample t-test:
$$d = \\frac{\\bar{X} - \\mu_0}{s}$$

#### Independent t-test:
$$d = \\frac{\\bar{X_1} - \\bar{X_2}}{s_{pooled}}$$

#### Paired t-test:
$$d = \\frac{\\bar{D}}{s_D}$$

### 🎯 Cohen's d 해석 기준

| Cohen's d | 효과크기 | 해석 | 예시 |
|-----------|----------|------|------|
| 0.0 - 0.2 | 매우 작음 | 실용적 의미 거의 없음 | 키 차이 1-2cm |
| 0.2 - 0.5 | 작음 | 작지만 의미있는 차이 | 시험점수 5-10점 차이 |
| 0.5 - 0.8 | 중간 | 육안으로 관찰 가능한 차이 | 체중 5-7kg 차이 |
| 0.8+ | 큼 | 명확하고 중요한 차이 | 치료 전후 극적 개선 |

### ⚡ 검정력 (Statistical Power)

**검정력**은 실제로 차이가 있을 때 이를 **올바르게 탐지할 확률**입니다.

**Power = 1 - β** (β는 Type II 오류율)

#### 검정력에 영향을 미치는 요소:
1. **효과크기** (클수록 검정력 ↑)
2. **표본크기** (클수록 검정력 ↑)
3. **유의수준 α** (클수록 검정력 ↑)
4. **분산** (작을수록 검정력 ↑)

In [ ]:
# 📊 효과크기와 검정력 분석
def effect_size_and_power_analysis():
    \"\"\"효과크기와 검정력 관계 분석\"\"\"
    
    # 다양한 효과크기에 따른 검정력 계산
    effect_sizes = np.arange(0, 2.1, 0.1)
    sample_sizes = [10, 20, 30, 50]
    alpha = 0.05
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            '1. 효과크기별 검정력 (One-sample)',
            '2. 표본크기별 검정력 비교',
            '3. 효과크기 해석 기준',
            '4. 실제 사례별 효과크기'
        ]
    )
    
    # 1. 효과크기별 검정력 곡선
    for n in sample_sizes:
        powers = []
        for d in effect_sizes:
            # One-sample t-test의 검정력 계산
            ncp = d * np.sqrt(n)  # Non-centrality parameter
            critical_t = stats.t.ppf(1 - alpha/2, n-1)
            
            # 검정력 = P(|T| > critical_t | H1 is true)
            power = 1 - (stats.nct.cdf(critical_t, n-1, ncp) - 
                        stats.nct.cdf(-critical_t, n-1, ncp))
            powers.append(power)
        
        fig.add_trace(
            go.Scatter(
                x=effect_sizes,
                y=powers,
                mode='lines',
                name=f'n = {n}',
                line=dict(width=2)
            ),
            row=1, col=1
        )
    
    # 검정력 0.8 기준선
    fig.add_hline(
        y=0.8,
        line_dash=\"dash\",
        line_color=\"red\",
        annotation_text=\"권장 검정력 (0.8)\",
        row=1, col=1
    )
    
    # 2. 특정 효과크기(d=0.5)에서 표본크기별 검정력
    sample_range = np.arange(5, 101, 5)
    d_medium = 0.5
    
    powers_medium = []
    for n in sample_range:
        ncp = d_medium * np.sqrt(n)
        critical_t = stats.t.ppf(1 - alpha/2, n-1)
        power = 1 - (stats.nct.cdf(critical_t, n-1, ncp) - 
                    stats.nct.cdf(-critical_t, n-1, ncp))
        powers_medium.append(power)
    
    fig.add_trace(
        go.Scatter(
            x=sample_range,
            y=powers_medium,
            mode='lines',
            name='d = 0.5 (중간 효과)',
            line=dict(width=3, color='blue')
        ),
        row=1, col=2
    )
    
    fig.add_hline(
        y=0.8,
        line_dash=\"dash\",
        line_color=\"red\",
        annotation_text=\"권장 검정력 (0.8)\",
        row=1, col=2
    )
    
    # 3. 효과크기 해석 기준
    effect_categories = ['매우 작음<br>(0.0-0.2)', '작음<br>(0.2-0.5)', 
                        '중간<br>(0.5-0.8)', '큼<br>(0.8+)']
    effect_ranges = [0.1, 0.35, 0.65, 1.0]
    colors = ['lightgray', 'yellow', 'orange', 'red']
    
    fig.add_trace(
        go.Bar(
            x=effect_categories,
            y=effect_ranges,
            marker_color=colors,
            name='효과크기 기준',
            text=effect_ranges,
            textposition='auto'
        ),
        row=2, col=1
    )
    
    # 4. 실제 사례별 효과크기 예시
    cases = ['키 차이\n(남녀)', '교육 효과\n(일반적)', '치료 효과\n(우울증)', '약물 효과\n(혈압)']
    case_effects = [1.4, 0.3, 0.8, 0.6]
    case_colors = ['red' if d >= 0.8 else 'orange' if d >= 0.5 else 'yellow' if d >= 0.2 else 'lightgray' 
                  for d in case_effects]
    
    fig.add_trace(
        go.Bar(
            x=cases,
            y=case_effects,
            marker_color=case_colors,
            name='실제 사례',
            text=[f'{d:.1f}' for d in case_effects],
            textposition='auto'
        ),
        row=2, col=2
    )
    
    # 레이아웃 업데이트
    fig.update_layout(
        title='📊 효과크기와 검정력: p-값을 넘어선 실질적 의미',
        height=800
    )
    
    # 축 레이블
    fig.update_xaxes(title_text='효과크기 (Cohen\'s d)', row=1, col=1)
    fig.update_yaxes(title_text='검정력', row=1, col=1)
    fig.update_xaxes(title_text='표본크기', row=1, col=2)
    fig.update_yaxes(title_text='검정력', row=1, col=2)
    fig.update_yaxes(title_text='Cohen\'s d', row=2, col=1)
    fig.update_yaxes(title_text='Cohen\'s d', row=2, col=2)
    
    return fig

# 효과크기와 검정력 분석 실행
fig_power = effect_size_and_power_analysis()
fig_power.show()

print(\"\"\"\n📊 효과크기와 검정력 핵심 통찰:\n
🎯 효과크기 (Cohen's d):\n
   • 통계적 유의성과는 별개의 실용적 중요도\n
   • 0.2 (작음), 0.5 (중간), 0.8 (큼) 기준\n
   • 분야별로 기준이 다를 수 있음\n
\n
⚡ 검정력 (Statistical Power):\n
   • 실제 차이를 올바르게 탐지할 확률\n
   • 일반적으로 0.8 이상 권장\n
   • 표본크기 설계의 핵심 요소\n
\n
💡 실용적 가이드라인:\n
   • p < 0.05: \"차이가 존재한다\"\n
   • Cohen's d ≥ 0.5: \"실용적으로 의미있다\"\n
   • Power ≥ 0.8: \"신뢰할 만한 설계다\"\n
\"\"\")

## 🧭 5. 올바른 t-검정 선택 가이드

### 🎯 결정 트리: 어떤 검정을 사용할까?

```
❓ 몇 개 그룹을 비교하나요?
├── 1개 그룹 → 🎯 One-sample t-test (아토스)
│   └── 알려진 기준값과 비교
└── 2개 그룹
    ├── ❓ 같은 개체의 전후 측정인가요?
    │   ├── 예 → 🔄 Paired t-test (아라미스)
    │   └── 아니오 → ⚖️ Independent t-test (포르토스)
    │       ├── ❓ 등분산인가요?
    │       ├── 예 → Student's t-test
    │       └── 아니오 → Welch's t-test
```

### 📋 체크리스트

#### 🎯 One-sample t-test를 선택하세요
- ✅ 하나의 그룹이 있음
- ✅ 알려진 기준값(모집단 평균)과 비교
- ✅ 예: "우리 제품이 규격에 맞나?", "평균이 100인가?"

#### ⚖️ Independent t-test를 선택하세요
- ✅ 두 개의 독립적인 그룹
- ✅ 서로 다른 개체들로 구성
- ✅ 예: "남성 vs 여성", "A약 vs B약", "서울 vs 부산"

#### 🔄 Paired t-test를 선택하세요
- ✅ 동일한 개체의 두 번 측정
- ✅ 전후 비교, 짝지어진 데이터
- ✅ 예: "치료 전후", "교육 전후", "수술 전후"

In [ ]:
# 🧭 올바른 t-검정 선택 시뮬레이터
def t_test_selector_quiz():
    \"\"\"대화형 t-검정 선택 가이드\"\"\"
    
    scenarios = [
        {
            'title': '🏥 신약 효과 테스트',
            'description': '새로운 혈압약을 20명의 환자에게 투약하고, 투약 전후의 혈압을 측정했습니다.',
            'correct_test': 'Paired t-test',
            'reason': '동일한 환자들의 투약 전후 비교이므로 Paired t-test를 사용합니다.',
            'hero': '아라미스'
        },
        {
            'title': '📱 배터리 수명 검사',
            'description': '스마트폰 배터리 30개를 테스트하여 평균 수명이 제조사 공칭값 24시간과 같은지 확인하고 싶습니다.',
            'correct_test': 'One-sample t-test',
            'reason': '하나의 그룹을 알려진 기준값(24시간)과 비교하므로 One-sample t-test를 사용합니다.',
            'hero': '아토스'
        },
        {
            'title': '🎓 학습 방법 비교',
            'description': '온라인 학습 그룹 25명과 오프라인 학습 그룹 30명의 시험 점수를 비교하고 싶습니다.',
            'correct_test': 'Independent t-test',
            'reason': '서로 다른 두 그룹(온라인 vs 오프라인)을 비교하므로 Independent t-test를 사용합니다.',
            'hero': '포르토스'
        },
        {
            'title': '🏃‍♀️ 운동 효과 분석',
            'description': '15명이 3개월 운동 프로그램에 참여하여 체지방률 변화를 측정했습니다.',
            'correct_test': 'Paired t-test',
            'reason': '동일한 사람들의 운동 전후 비교이므로 Paired t-test를 사용합니다.',
            'hero': '아라미스'
        },
        {
            'title': '☕ 지역별 커피 가격',
            'description': '강남구 카페 20곳과 강북구 카페 18곳의 아메리카노 가격을 비교하고 싶습니다.',
            'correct_test': 'Independent t-test',
            'reason': '서로 다른 두 지역의 독립적인 카페들을 비교하므로 Independent t-test를 사용합니다.',
            'hero': '포르토스'
        }
    ]
    
    print(\"🧭 t-검정 선택 마스터 퀴즈!\")
    print(\"각 상황에 맞는 올바른 t-검정을 선택해보세요.\\n\")
    
    score = 0
    total = len(scenarios)
    
    for i, scenario in enumerate(scenarios, 1):
        print(f\"📝 문제 {i}: {scenario['title']}\")
        print(f\"상황: {scenario['description']}\")
        print(\"\")
        print(\"선택지:\")
        print(\"1) 🎯 One-sample t-test (아토스)\")
        print(\"2) ⚖️ Independent t-test (포르토스)\")
        print(\"3) 🔄 Paired t-test (아라미스)\")
        print(\"\")
        
        # 정답 공개
        print(f\"✅ 정답: {scenario['correct_test']} ({scenario['hero']})\")
        print(f\"💡 이유: {scenario['reason']}\")
        print(\"\" + \"=\" * 60)
        print(\"\")
    
    # 요약 테이블 생성
    summary_df = pd.DataFrame([
        {
            '검정 유형': '🎯 One-sample t-test',
            '영웅': '아토스',
            '상황': '1개 그룹 vs 기준값',
            '예시': '제품 품질, 평균 비교',
            '공식': '(X̄ - μ₀) / (s/√n)'
        },
        {
            '검정 유형': '⚖️ Independent t-test',
            '영웅': '포르토스',
            '상황': '2개 독립 그룹 비교',
            '예시': 'A/B 테스트, 성별 차이',
            '공식': '(X̄₁ - X̄₂) / SE_pooled'
        },
        {
            '검정 유형': '🔄 Paired t-test',
            '영웅': '아라미스',
            '상황': '동일 개체 전후 비교',
            '예시': '치료 효과, 교육 효과',
            '공식': 'D̄ / (s_D/√n)'
        }
    ])
    
    print(\"📊 t-검정 삼총사 요약표:\")
    print(summary_df.to_string(index=False))
    
    return summary_df

# t-검정 선택 가이드 실행
summary_table = t_test_selector_quiz()

print(\"\"\"\n\n🎉 t-검정 삼총사 완전 정복!\n
💡 기억해야 할 핵심 포인트:\n
🎯 아토스 (One-sample): \"내가 기준에 맞나?\"\n
⚖️ 포르토스 (Independent): \"둘이 서로 다른가?\"\n
🔄 아라미스 (Paired): \"나에게 변화가 있었나?\"\n
\n
🔍 분석 순서:\n
1️⃣ 상황 파악 → 올바른 검정 선택\n
2️⃣ 가정 확인 → 등분산성 검정 등\n
3️⃣ 검정 실행 → t-통계량, p-값 계산\n
4️⃣ 효과크기 → Cohen's d로 실용적 의미 평가\n
5️⃣ 결과 해석 → 통계적 + 실용적 의미 종합\n
\"\"\")

## 📚 핵심 개념 요약

### ⚔️ t-test 삼총사의 특별한 능력

1. **🎯 아토스 (One-sample t-test)**
   - **임무**: 단일 그룹을 기준값과 비교
   - **공식**: $t = \\frac{\\bar{X} - \\mu_0}{s/\\sqrt{n}}$
   - **활용**: 품질 관리, 기준 준수 확인

2. **⚖️ 포르토스 (Independent t-test)**
   - **임무**: 두 독립 그룹 간 차이 검정
   - **공식**: $t = \\frac{\\bar{X_1} - \\bar{X_2}}{s_p\\sqrt{\\frac{1}{n_1} + \\frac{1}{n_2}}}$
   - **활용**: A/B 테스트, 그룹 간 비교

3. **🔄 아라미스 (Paired t-test)**
   - **임무**: 동일 개체의 전후 변화 검정
   - **공식**: $t = \\frac{\\bar{D}}{s_D/\\sqrt{n}}$
   - **활용**: 치료 효과, 교육 효과 분석

### 📊 효과크기와 검정력

- **Cohen's d**: 실용적 중요도 측정 (0.2/0.5/0.8 기준)
- **검정력**: 실제 차이를 탐지할 확률 (0.8 이상 권장)
- **p-값의 한계**: 통계적 유의성 ≠ 실용적 중요성

### 🎯 선택 기준

| 상황 | 그룹 수 | 데이터 구조 | 선택할 검정 |
|------|---------|-------------|-------------|
| 기준값과 비교 | 1개 | 단일 측정 | One-sample |
| 두 그룹 비교 | 2개 | 독립적 | Independent |
| 전후 비교 | 2개 측정 | 짝지어짐 | Paired |

---

## 🧩 연습 문제

### 문제 1: 검정 선택
다음 상황에서 어떤 t-검정을 사용해야 할까요?
- a) 새로운 다이어트약을 20명에게 투여하고 체중 변화를 측정
- b) 남학생 30명과 여학생 25명의 수학 점수 비교
- c) 공장에서 생산된 볼트 50개의 길이가 10cm 규격에 맞는지 확인

### 문제 2: 효과크기 해석
Cohen's d = 0.3인 경우, 이 효과크기를 어떻게 해석해야 할까요?

### 문제 3: 검정력 분석
효과크기가 0.5일 때, 검정력 0.8을 달성하려면 최소 몇 명의 표본이 필요할까요?

---

## 🚀 다음 여행지: "완벽하지 않은 현실"

삼총사의 능력을 익혔으니, 이제 **현실의 복잡함**을 다룰 차례입니다! 🌪️

다음 노트북에서는:
- **t-검정의 가정들**: 정규성, 독립성, 등분산성
- **가정 위반 시 대처법**: Welch's t-test, 비모수 검정
- **B.L. Welch의 혁신**: 등분산 가정 없는 t-검정
- **Satterthwaite 자유도**: 복잡한 현실을 위한 수학
- **강건성과 민감성**: 언제 걱정해야 할까?

현실은 완벽하지 않지만, 우리에게는 대처법이 있습니다! 💪

**다음 노트북**: `04_assumptions_and_violations.ipynb`